In [1]:
from bs4 import BeautifulSoup
import requests
import mysql.connector
from mysql.connector import errorcode

In [88]:
cnx = mysql.connector .connect(
    host = 'serverandres.c43rrwosnqfo.us-east-2.rds.amazonaws.com',
    user = 'owen_peng',
    password = 'Peng1234!'
)

In [36]:
page = requests.get("https://reelgood.com/tv/source/hulu?filter-sort=2&offset=650")
soup = BeautifulSoup(page.content, 'html.parser')
type(soup)

bs4.BeautifulSoup

In [60]:
body = soup.tbody

In [67]:
rows = body.find_all(class_="css-78jh1y")

In [71]:
rows[0].text

'The Carmichael Show'

In [74]:
def get_names(url):   
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    pictures = soup.tbody.find_all(class_="css-78jh1y")
    show_names = []
    for i in pictures:
        show_name = i.text
        show_names.append(show_name)
    return show_names

In [77]:
action = get_names('https://reelgood.com/tv/source/hulu?filter-genre=5&filter-sort=2')

In [76]:
api_key = '9cd2dde4'
url = 'http://www.omdbapi.com/?apikey=9cd2dde4&'

In [78]:
action[0]

'Rick and Morty'

In [79]:
def api_call(l):
    raw_show_data = []
    for i in range(0,len(l)):
        params = {'s':l[i]}
        response = requests.get(url, params=params)
        data = response.json()
        raw_show_data.append(data)
    return raw_show_data

In [83]:
def parse_raw_data(l,genre):
    parsed_data = []
    for i in l:
        if i['Response'] == 'False':
            continue
        else:
            try:
                movie_tuple = (i['Search'][0]['Title'],i['Search'][0]['Year'].split("–")[0],i['Search'][0]['imdbID'],genre)
            except:
                movie_tuple = (i['Search'][0]['Title'],i['Search'][0]['Year'],i['Search'][0]['imdbID'],genre)
            parsed_data.append(movie_tuple)
    return parsed_data

In [92]:
def db_entry(l,genre):
    raw_data = api_call(l)
    parsed_data = parse_raw_data(raw_data,genre)
    stmt = """INSERT INTO Hulu (Name, Start_Year, imdbID, Genre) VALUES (%s, %s, %s, %s)"""
    cursor.executemany(stmt, parsed_data)
    cnx.commit()

In [97]:
action = get_names('https://reelgood.com/tv/source/hulu?filter-genre=5&offset=100')

In [98]:
db_entry(action, 'Action')

In [100]:
animation = get_names('https://reelgood.com/tv/source/hulu?filter-genre=6')

In [ ]:
db_entry(animation, 'Animation')

In [89]:
cursor = cnx.cursor()

In [90]:
cursor.execute('''USE TVSHOWS''')

In [91]:
create=  (  "CREATE TABLE Hulu ("
        "  id int NOT NULL AUTO_INCREMENT,"
        "  Name varchar(100),"
        "  Start_Year int,"
        "  imdbID varchar(50) NOT NULL,"
        "  genre varchar(50),"  
        "  PRIMARY KEY (iD)"
        ") ENGINE=InnoDB")
cursor.execute(create)